# F2 Quantum Fourier Transform

In the previous node we discussed that the DFT matrix is unitary, so could we benefit from a quantum implementation of the Fourier transform? In this node, we discuss the basics of the Quantum Fourier transform (QFT) and figure out how we can go about implementing it. The QFT matrix looks very similar to the DFT matrix except with a normalization factor of . 

Codercise F.2.1. Implement the circuit that performs the single-qubit QFT operation, i.e., for .

In [4]:
import numpy as np
import pennylane as qml

In [5]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def one_qubit_QFT(basis_id):
    """A circuit that computes the QFT on a single qubit. 
    
    Args:
        basis_id (int): An integer value identifying 
            the basis state to construct.
    
    Returns:
        array[complex]: The state of the qubit after applying QFT.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    qml.BasisStatePreparation(bits, wires=[0])

    ##################
    # YOUR CODE HERE #
    ##################
    
    qml.Hadamard(wires=0)
    return qml.state()


The QFT circuit for a single qubit is simply the Hadamard gate! Let us now implement the two-qubit QFT.

Codercise F.2.2. Implement a circuit that performs the two-qubit QFT operation. First, derive the QFT matrix for . Then, use the qml.QubitUnitary operation to apply it.

In [10]:
n_bits = 2
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def two_qubit_QFT(basis_id):
    """A circuit that computes the QFT on two qubits using qml.QubitUnitary. 
    
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
    
    Returns:
        array[complex]: The state of the qubits after the QFT operation.
    """
    
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    qml.BasisStatePreparation(bits, wires=[0, 1])
    
    ##################
    # YOUR CODE HERE #
    ##################

    U = 1 / 2 * np.array([[1, 1, 1, 1], 
                          [1, complex(0,1), -1, complex(0,-1)],
                          [1, -1, 1, -1],
                          [1, complex(0,-1), -1, complex(0,1)]
                          ])

    qml.QubitUnitary(U, wires=[0,1])

    return qml.state()

If we were going to actually do this on a quantum computer, we'd need to express it in terms of elementary gates. In general, this is the problem of circuit synthesis; it turns out, though, that the QFT has a well-defined, recursive structure, which you will start to see in the next exercise.

Let's look at the two-qubit QFT matrix:

Codercise F.2.3. Implement the two-qubit QFT using a combination of gates (without using qml.QubitUnitary).

Hint.
Here is a PennyLane function that you may not have encountered until now. A gate can be transformed into a controlled gate by applying qml.ctrl.

For example,

qml.ctrl(qml.PauliY, control=1)(wires=0)
is equivalent to applying a controlled- operation where qubit 1 is the control, and qubit 0 is the target.

Hint.
Don't forget to swap the inner rows back!

In [11]:
# qml.ctrl(qml.PauliY, control=1)(wires=0)

In [12]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def decompose_two_qubit_QFT(basis_id):
    """A circuit that computes the QFT on two qubits using elementary gates.
    
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
    
    Returns:
        array[complex]: The state of the qubits after the QFT operation.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    qml.BasisStatePreparation(bits, wires=[0, 1])
    
    ##################
    # YOUR CODE HERE #
    ##################
    
    qml.Hadamard(wires=0)
    
    qml.ctrl(qml.S, control=1)(wires=0)

    qml.Hadamard(wires=1)

    qml.SWAP(wires=[0,1])
    
    return qml.state()